In [1]:
import geopandas as gpd
import pandas as pd

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
data_pop=gpd.read_file('data/output/1_population_density/grid_pop_19_100m.gpkg')
data_degurba_grod=gpd.read_file('data/output/2_grid_degurba/degurba_1900.gpkg')

In [3]:
data_desa_raster=gpd.read_file('data/output/0_raster_data_desa/desa_pend_rasterize_19.gpkg')

In [4]:
pop_w_grid=data_pop.sjoin(data_degurba_grod)[['j_pend','p_builtup','classification','geometry']]

In [5]:
data_desa=data_desa_raster.sjoin(pop_w_grid)

In [6]:
data_desa

,iddesa,DN,id,pop2020,geometry,index_right,j_pend,p_builtup,classification
0,1901070002,1.0,1,1087,"POLYGON ((10609800.000 -277100.000, 10610500.0...",306734,0.000000,0.000072,RURAL
243,1904030015,246.0,246,1955,"MULTIPOLYGON (((10596600.000 -296500.000, 1059...",306734,0.000000,0.000072,RURAL
0,1901070002,1.0,1,1087,"POLYGON ((10609800.000 -277100.000, 10610500.0...",307251,0.000000,0.000000,RURAL
243,1904030015,246.0,246,1955,"MULTIPOLYGON (((10596600.000 -296500.000, 1059...",307251,0.000000,0.000000,RURAL
0,1901070002,1.0,1,1087,"POLYGON ((10609800.000 -277100.000, 10610500.0...",307147,0.000000,0.000000,RURAL
...,...,...,...,...,...,...,...,...,...
388,1971041006,391.0,391,8384,"POLYGON ((10628200.000 -260000.000, 10628700.0...",792670,74.214074,0.364982,CENTRE
388,1971041006,391.0,391,8384,"POLYGON ((10628200.000 -260000.000, 10628700.0...",792670,74.214074,0.385393,CENTRE
388,1971041006,391.0,391,8384,"POLYGON ((10628200.000 -260000.000, 10628700.0...",792661,26.352242,0.385393,CENTRE
388,1971041006,391.0,391,8384,"POLYGON ((10628200.000 -260000.000, 10628700.0...",792661,26.352242,0.364982,CENTRE


In [7]:
tab_data=pd.pivot_table(data_desa,index='iddesa',columns='classification',values='j_pend',aggfunc='sum').fillna(0).reset_index()

In [8]:
def define_LAU(y):
    total_pop=y.CENTRE+y.CLUSTER+y.RURAL
    if total_pop==0:
        return 'RURALS'
    elif (y.CENTRE/total_pop)>=0.5:
        return 'CITIES'
    elif (((y.CENTRE/total_pop)<0.5)&((y.RURAL/total_pop)<0.5)):
        return 'TOWNS'
    else:
        return 'RURALS'

In [9]:
tab_data['LAU']=tab_data.apply(lambda y:define_LAU(y),axis=1)

In [10]:
tab_data

classification,iddesa,CENTRE,CLUSTER,RURAL,LAU
0,1901070002,0.000000,0.000000,1494.227103,RURALS
1,1901070003,0.000000,2130.056162,372.895592,TOWNS
2,1901070004,0.000000,4910.231426,5925.007288,RURALS
3,1901070005,1417.705227,2983.276480,1493.897544,TOWNS
4,1901070006,0.000000,1097.532412,886.918386,TOWNS
...,...,...,...,...,...
384,1971041002,23485.223660,0.000000,0.000000,CITIES
385,1971041003,11380.998171,2272.837846,1658.899794,CITIES
386,1971041004,12866.658122,0.000000,0.000000,CITIES
387,1971041005,9998.099926,0.000000,0.000000,CITIES


In [11]:
kdprov='19'
peta_desa=gpd.read_file(r"data/input/Master_Data/DESA_20201_BYINDO.gpkg").query('kdprov==@kdprov')

In [12]:
peta_desa_LAU=peta_desa.merge(tab_data[['iddesa','LAU']],how='left').fillna('RURALS')

In [13]:
peta_desa_LAU.to_file('data/output/3_small_spatial_unit_degurba/small_spatial_unit_'+kdprov+'.gpkg',driver='GPKG')

In [16]:
from dbfread import DBF
mfd_=pd.DataFrame(iter(DBF('data/input/Master_Data/mfd_2020_1.dbf')))

In [19]:
mfd_['iddesa']=mfd_.PROP+mfd_.KAB+mfd_.KEC+mfd_.DESA

In [22]:
peta_desa_LAU.merge(mfd_[['iddesa','DRH']]).to_file('data/output/3_small_spatial_unit_degurba/small_spatial_unit_'+kdprov+'.gpkg',driver='GPKG')

In [26]:
peta_desa_LAU.merge(mfd_[['iddesa','DRH']]).pivot_table(index='DRH',columns='LAU',values='iddesa',aggfunc='count')

LAU,CITIES,RURALS,TOWNS
DRH,,,
1,47,22,61
2,3,204,56


In [29]:
peta_desa_LAU.merge(mfd_[['iddesa','DRH']]).query('LAU==\'CITIES\'').query('DRH==\'2\'')

,iddesa,idkab,idkec,nmprov,nmkab,nmkec,nmdesa,kdprov,kdkab,kdkec,kddesa,sumber,periode,geometry,LAU,DRH
220,1904020010,1904,1904020,KEPULAUAN BANGKA BELITUNG,BANGKA TENGAH,PANGKALAN BARU,MANGKOL,19,04,020,010,BPS,2020_1,"POLYGON ((106.08995 -2.14536, 106.08997 -2.145...",CITIES,2
381,1971031006,1971,1971031,KEPULAUAN BANGKA BELITUNG,PANGKALPINANG,GABEK,JERAMBAH GANTUNG,19,71,031,006,BPS,2020_1,"POLYGON ((106.09903 -2.07681, 106.09946 -2.077...",CITIES,2
388,1971041003,1971,1971041,KEPULAUAN BANGKA BELITUNG,PANGKALPINANG,GERUNGGANG,TUATUNU INDAH,19,71,041,003,BPS,2020_1,"POLYGON ((106.06206 -2.06822, 106.06252 -2.068...",CITIES,2
